In [96]:
import PyPDF2
import re

In [97]:
file_path = 'plan5.pdf'

pdfFileObj = open(file_path, 'rb') 
        
# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

page_num = pdfReader.numPages
pages_text = []
for index in range(page_num):
    page = pdfReader.getPage(index)
    pages_text.append(page.extractText())

# extracting text from page 
#print(info) 
    
# closing the pdf file object 
pdfFileObj.close() 

In [98]:
def get_subjects(text) -> str:
    pattern = r'.*(?=Wykonał:)'
    return re.findall(pattern, text, flags=re.DOTALL)[0]

In [99]:
get_subjects(page_1)

NameError: name 'page_1' is not defined

In [ ]:
pattern = r".*(?=Wykonał:)"
out = None
for match in re.finditer(pattern, page_1, re.DOTALL):
    out = match.group(2)

IndexError: no such group

In [ ]:
def plan_author(text) -> str:
    pattern = r'Dziekan:(.*?)(\w+\s[A-Z][a-z]+)(?=[A-Z])'
    return re.findall(pattern, text, flags=re.DOTALL)[0][-1]

In [ ]:
def get_semestr(text) -> str:
    pattern = r'(=?semestr\s)(\d+)'
    return re.findall(pattern, text, flags=re.DOTALL)[0][-1]

In [ ]:
get_semestr(page_1)

'5'

In [ ]:
plan_author(page_1)

'Mariusz Bodzioch'

In [ ]:
def get_date(text, *, week_name: str) -> str:
    """Get date by given week name"""
    pattern = fr'(=?{week_name}\s)([0-9.]+)'
    return re.findall(pattern, text, flags=re.DOTALL)[0][-1]

In [ ]:
get_date(page_1, week_name='Sobota')

'10.10.2020'

In [ ]:
get_date(page_1, week_name='Niedziela')

'11.10.202013'

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd
import enum
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
import pdfminer

# Open a PDF file.
fp = open('plan5.pdf', 'rb')

# Create a PDF parser object associated with the file object.
parser = PDFParser(fp)

# Create a PDF document object that stores the document structure.
# Password for initialization as 2nd parameter
document = PDFDocument(parser)

# Check if the document allows text extraction. If not, abort.
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed

# Create a PDF resource manager object that stores shared resources.
rsrcmgr = PDFResourceManager()

# Create a PDF device object.
device = PDFDevice(rsrcmgr)

# BEGIN LAYOUT ANALYSIS
# Set parameters for analysis.
laparams = LAParams()

# Create a PDF page aggregator object.
device = PDFPageAggregator(rsrcmgr, laparams=laparams)

# Create a PDF interpreter object.
interpreter = PDFPageInterpreter(rsrcmgr, device)

info = []

def parse_obj(lt_objs, index):

    # loop over the object list
    for obj in lt_objs:

        # if it's a textbox, print text and location
        if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
            print ("%6d, %6d, %s" % (obj.bbox[0], obj.bbox[1], obj.get_text().replace('\n', '_')))
            info.append((obj.bbox[0], obj.bbox[1], obj.get_text().replace('\n', '_'), index))
        # if it's a container, recurse
        elif isinstance(obj, pdfminer.layout.LTFigure):
            parse_obj(obj._objs)

# loop over all pages in the document
for i, page in enumerate(PDFPage.create_pages(document)):

    # read the page into a layout object
    interpreter.process_page(page)
    layout = device.get_result()

    # extract text from this object
    parse_obj(layout._objs, i)

In [115]:
def get_hour_from_cord_tekst(val: str) -> str:
    if len(hour_definition[hour_definition.x <= val]):
        return hour_definition[hour_definition.x <= val].iloc[-1].Przedmiot
    return -1
def get_hour_from_cord_x(val: str) -> str:
    if len(hour_definition[hour_definition.x <= val]):
        return hour_definition[hour_definition.x <= val].iloc[-1].x
    return -1
def map_hour(hour: str, diff: int):
    if len(str(hour)) < 3:
        return -1
    hour = int(hour[:2])
    if diff > 38:
        return hour + 1
    if diff < 13:
        return f'{hour}:15'
    if diff < 26:
        return f'{hour}:30'
    return f'{hour}:45'

arr = np.array(info)
df = pd.DataFrame(arr)
df.columns = ['x', 'y', 'Przedmiot', 'Strona']
df.x = df.x.astype('float')
df.y = df.y.astype('float')
df

hour_definition = df[(df['Strona'] == '1') & (df['Przedmiot'].str.contains('00_'))]
hour_definition.drop_duplicates(subset=['Przedmiot'], inplace=True)
hour_definition.reset_index(drop=True, inplace=True)
hour_definition

c:\Users\Krystian\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,x,y,Przedmiot,Strona
0,85.80,473.5816,07:00_,1
1,137.18,473.5816,08:00_,1
2,188.54,473.5816,09:00_,1
3,239.90,473.5816,10:00_,1
4,291.29,473.5816,11:00_,1
5,342.65,473.5816,12:00_,1
6,394.01,473.5816,13:00_,1
7,445.39,473.5816,14:00_,1
8,496.75,473.5816,15:00_,1
9,548.11,473.5816,16:00_,1


In [139]:
hour = hour_definition.iloc[:2,:].x
hour_diff = int(hour[1] - hour[0])
hour_diff

51

In [195]:
df_tmp = df[df['Strona'] == '1']
def map_under_text(row):
    df_with_text = df_tmp[(
        df_tmp.Przedmiot.str.contains('\[') == False) 
        & (df_tmp.y < row.y) 
        & (df_tmp.y > row.y - 30) 
        & (df_tmp.x > row.x - hour_diff//2) 
        & (df_tmp.x < row.x + hour_diff)
    ].append(row).sort_index()
    return ' '.join(df_with_text.Przedmiot.values).replace('_', '')

In [205]:
df_tmp['hour_mapped'] = list(map(get_hour_from_cord_tekst, list(df_tmp.x)))
df_tmp['x_cord_mapped'] = list(map(get_hour_from_cord_x, list(df_tmp.x)))
df_tmp['hour_diff'] = df_tmp.x - df_tmp.x_cord_mapped
df_tmp['Godzina rozpoczęcia'] = df_tmp.apply(lambda x: map_hour(x['hour_mapped'], x['hour_diff']), axis=1)
df_tmp.Przedmiot = df_tmp.apply(lambda x: map_under_text(x), axis=1)
df_tmp[df_tmp.Przedmiot.str.contains('\[') == True][['Przedmiot', 'Strona', 'Godzina rozpoczęcia']]

C:\Users\Krystian\AppData\Local\Temp/ipykernel_4468/3787455264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['hour_mapped'] = list(map(get_hour_from_cord_tekst, list(df_tmp.x)))
C:\Users\Krystian\AppData\Local\Temp/ipykernel_4468/3787455264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['x_cord_mapped'] = list(map(get_hour_from_cord_x, list(df_tmp.x)))
C:\Users\Krystian\AppData\Local\Temp/ipykernel_4468/3787455264.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,Przedmiot,Strona,Godzina rozpoczęcia
60,"ME [w], dr hab. L.Błaszkiewicz, sala C2 L.Bł...",1,13:15
61,"ME [w], dr hab. L.Błaszkiewicz, sala C2 L.Bł...",1,15
66,"WdGM [w], dr hab. A.Denisiuk,sala C2",1,16:45
67,"WdGM [w], dr hab. A.Denisiuk,sala C2",1,18:30
83,"IO [w], dr M.Kolev, sala C2 sala C2 sala C2 s...",1,8
84,"IO [w], dr M.Kolev, sala C2 sala C2 sala C2 s...",1,9:45
87,"ME [w], dr hab. L.Błaszkiewicz, sala C2 L.Bł...",1,11:30
88,"ME [w], dr hab. L.Błaszkiewicz, sala C2 L.Bł...",1,13:15
93,"ERI [w], dr inż. B.Nowak, sala C2",1,15
94,"WWW [w], dr inż. B.Nowak, sala C2",1,16:45


In [ ]:
df_tmp.to_csv('output1.csv', index=False)